In [1]:

import pandas as pd
import numpy as np
# from tqdm import tqdm
# from tqdm import tnrange, tqdm_notebook
# from time import sleep
import difflib
import scipy
import operator
import difflib
from IPython.core.display import display,HTML
from tqdm import tnrange, tqdm_notebook

try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle



display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [5]:
path="/home/juliaponcela/at_NICO/Supply_networks/Results/"
filename="Simplified_stata_file_Pinf_vs_mulitiple_variables_monthly_slicing_dropped_overlaps_randomly_from_list.csv"

df= pd.read_csv(path +filename, sep=' ',low_memory=False)

print df.shape
df.head()

(282164, 64)


,P_inf,Mi,Cj,Mi_Cj,MCJ,num_trans,start_date_trans,end_date_trans,first_day_trans,last_day_trans,...,min_dist_j_to_inf,avg_dist_j_to_inf,degree_asym_ij,business_asym_ij,zs_error_rate_ij_year_degree,Dist,zip1,zip2,core_score_year_i,core_score_year_j
0,1,100055,2300055,100055_2300055,4,1,1987-05-01,1987-05-31,850,880,...,1,4.068182,0.404959,NaN,4.596074,0.343654,10018.0,10001,0.222222,0.333333
1,0,100055,400055,100055_400055,4,1,1987-05-01,1987-05-30,850,879,...,3,6.068182,0.111111,0.000000,-0.223313,0.838389,10018.0,10019,0.222222,0.333333
2,0,100286,1000019,100286_1000019,1,2,1985-03-01,1985-03-31,59,89,...,1,2.065934,0.990099,0.997706,-0.361477,NaN,NaN,NaN,1.000000,0.873899
3,1,100365,1000019,100365_1000019,1,4,1985-01-01,1985-12-31,0,364,...,1,2.348000,0.957224,0.954538,3.142229,NaN,10001.0,NaN,1.000000,0.873899
4,0,100365,1000019,100365_1000019,1,1,1986-01-01,1986-06-30,365,545,...,1,2.108108,0.978083,0.985517,-0.442647,NaN,10001.0,NaN,0.583333,0.835772


In [7]:
df=df.sort_values(by=['Mi_Cj', 'start_date_trans'], ascending=[1, 1])

df['P_death'] = 0 # i add a new column  (it will indicate the last instance of a relationship)

In [47]:
# df[df['Mi_Cj'] == "100365_1000019"]

,P_inf,Mi,Cj,Mi_Cj,MCJ,num_trans,start_date_trans,end_date_trans,first_day_trans,last_day_trans,...,avg_dist_j_to_inf,degree_asym_ij,business_asym_ij,zs_error_rate_ij_year_degree,Dist,zip1,zip2,core_score_year_i,core_score_year_j,P_death
3,1,100365,1000019,100365_1000019,1,4,1985-01-01,1985-12-31,0,364,...,2.348000,0.957224,0.954538,3.142229,NaN,10001.0,NaN,1.000000,0.873899,0
4,0,100365,1000019,100365_1000019,1,1,1986-01-01,1986-06-30,365,545,...,2.108108,0.978083,0.985517,-0.442647,NaN,10001.0,NaN,0.583333,0.835772,0
5,0,100365,1000019,100365_1000019,1,1,1986-08-01,1986-08-31,577,607,...,2.190000,0.976191,0.983562,-0.442647,NaN,10001.0,NaN,0.583333,0.835772,0
6,0,100365,1000019,100365_1000019,1,1,1986-10-01,1986-12-31,638,729,...,2.313008,0.962268,0.987817,-0.442647,NaN,10001.0,NaN,0.583333,0.835772,0
7,0,100365,1000019,100365_1000019,1,1,1987-01-01,1987-04-30,730,849,...,2.215385,0.961787,0.936574,-0.205185,NaN,10001.0,NaN,0.645833,0.843040,0


In [34]:
list_ids=df.Mi_Cj.unique()


In [45]:
# for mi_cj in list_ids:
#     selection = df[df['Mi_Cj'] == mi_cj]
#     #print mi_cj, selection.shape
#     #raw_input()

In [8]:
# def add_Pdeath(input_df):
    
   
#    # print input_df#.shape,len(input_df)
#     #print "index",input_df.index[-1]
    
#     index_last= input_df.index[-1]    #### Int64Index([37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], dtype='int64')    
#     input_df.set_value(index_last, 'P_death', 1)
        
        
        
#     #print input_df
# #     print 
# #     print 
# #     raw_input()
#     return input_df


# new_df = df.groupby('Mi_Cj').apply(add_Pdeath)

#grouped= grouped.reset_index()  # to reset the indexes (instead of using the authour_id as index, i create a new one)



df['P_death'] = (df.Mi_Cj != df.Mi_Cj.shift(-1)).astype(int)   # shifting is a way of checking whether the value of a cell in a given row is is the same as 1,2,3,... rows before/after

In [9]:
# a faster way of doing the same:

df[['Mi_Cj',"P_death"]].tail(1000)

,Mi_Cj,P_death
281164,9106066_5243521,1
281165,9106066_5246342,1
281166,9106066_5258146,0
281167,9106066_5258146,0
281168,9106066_5258146,0
281169,9106066_5258146,1
281170,9106066_5260337,0
281171,9106066_5260337,0
281172,9106066_5260337,0
281173,9106066_5260337,0


In [10]:

writer = pd.ExcelWriter(path+filename.strip(".csv")+"_Pdeath.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

print "xlsx done:",path+filename.strip(".csv")+"_Pdeath.xlsx"





xlsx done: /home/juliaponcela/at_NICO/Supply_networks/Results/Simplified_stata_file_Pinf_vs_mulitiple_variables_monthly_slicing_dropped_overlaps_randomly_from_list_Pdeath.xlsx


In [11]:

df.to_csv(path+filename.strip(".csv")+"_Pdeath.tsv", sep='\t', encoding='utf-8')#, columns = list_headers)
print "csv done:", path+filename.strip(".csv")+"_Pdeath.tsv"

csv done: /home/juliaponcela/at_NICO/Supply_networks/Results/Simplified_stata_file_Pinf_vs_mulitiple_variables_monthly_slicing_dropped_overlaps_randomly_from_list_Pdeath.tsv
